### Tasks:
1. Use the RANK() function to assign a ranking number to each row based on the order specified within the window.

2. Use aggregate window functions to calculate running totals.

3. Use the LAG() function to help calculate the difference, in days, between consecutive date readings in our dataset.

4. Use aggregate window functions to calculate the moving average.

In [2]:
%load_ext sql

In [4]:
%sql sqlite:///northwind.db

#### Task 1

Rank all the orders of a specific customer from the most recent to the least recent using window functions. Assume that the customer ID is 'ALFKI'.

In [8]:
%%sql

SELECT
    *
FROM
    orders
WHERE
    customerid='ALFKI';

 * sqlite:///northwind.db
Done.


OrderID,CustomerID,EmployeeID,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
10643,ALFKI,6,1997-08-25 00:00:00,1997-09-22 00:00:00,1997-09-02 00:00:00,1,29.46,Alfreds Futterkiste,Obere Str. 57,Berlin,None,12209,Germany
10692,ALFKI,4,1997-10-03 00:00:00,1997-10-31 00:00:00,1997-10-13 00:00:00,2,61.02,Alfred-s Futterkiste,Obere Str. 57,Berlin,None,12209,Germany
10702,ALFKI,4,1997-10-13 00:00:00,1997-11-24 00:00:00,1997-10-21 00:00:00,1,23.94,Alfred-s Futterkiste,Obere Str. 57,Berlin,None,12209,Germany
10835,ALFKI,1,1998-01-15 00:00:00,1998-02-12 00:00:00,1998-01-21 00:00:00,3,69.53,Alfred-s Futterkiste,Obere Str. 57,Berlin,None,12209,Germany
10952,ALFKI,1,1998-03-16 00:00:00,1998-04-27 00:00:00,1998-03-24 00:00:00,1,40.42,Alfred-s Futterkiste,Obere Str. 57,Berlin,None,12209,Germany
11011,ALFKI,3,1998-04-09 00:00:00,1998-05-07 00:00:00,1998-04-13 00:00:00,1,1.21,Alfred-s Futterkiste,Obere Str. 57,Berlin,None,12209,Germany


In [9]:
%%sql

SELECT
    OrderID,
    OrderDate,
    RANK() OVER( 
        ORDER BY OrderDate DESC) AS OrderRank
FROM
    orders
WHERE
    customerid='ALFKI';

 * sqlite:///northwind.db
Done.


OrderID,OrderDate,OrderRank
11011,1998-04-09 00:00:00,1
10952,1998-03-16 00:00:00,2
10835,1998-01-15 00:00:00,3
10702,1997-10-13 00:00:00,4
10692,1997-10-03 00:00:00,5
10643,1997-08-25 00:00:00,6


#### Task 2
Calculate a running total of the quantity of orders using window functions.

In [21]:
%%sql

SELECT
    *
FROM
    orderDetails
ORDER BY 
    OrderID
LIMIT 5;

 * sqlite:///northwind.db
Done.


OrderID,ProductID,UnitPrice,Quantity,Discount
10248,11,14,12,0.0
10248,42,9.8,10,0.0
10248,72,34.8,5,0.0
10249,14,18.6,9,0.0
10249,51,42.4,40,0.0


In [23]:
%%sql

SELECT
    OrderID,
    Quantity,
    SUM(Quantity) OVER (
        ORDER BY OrderID) AS RunningTotal
FROM
    OrderDetails
GROUP BY
    OrderID
LIMIT 10;

 * sqlite:///northwind.db
Done.


OrderID,Quantity,RunningTotal
10248,12,12
10249,9,21
10250,10,31
10251,6,37
10252,40,77
10253,20,97
10254,15,112
10255,20,132
10256,15,147
10257,25,172


#### Task 3
Use window functions to find the difference in successive order dates for each customer. HINT: The TIMESTAMPDIFF() function in MySQL is not available in SQLite. We can use the julianday() function to convert the dates to a floating point number and then calculate the difference.

In [30]:
%%sql

SELECT
    OrderID,
    CustomerID,
    OrderDate,
    LAG(OrderDate,1) OVER (
        PARTITION BY CustomerID
        ORDER BY OrderDate) AS PrevOrderDate,
    julianday(OrderDate)-julianday(LAG(OrderDate,1) OVER (
        PARTITION BY CustomerID
        ORDER BY OrderDate)) AS DiffOrderDate
FROM
    Orders;

 * sqlite:///northwind.db
Done.


OrderID,CustomerID,OrderDate,PrevOrderDate,DiffOrderDate
10643,ALFKI,1997-08-25 00:00:00,None,None
10692,ALFKI,1997-10-03 00:00:00,1997-08-25 00:00:00,39.0
10702,ALFKI,1997-10-13 00:00:00,1997-10-03 00:00:00,10.0
10835,ALFKI,1998-01-15 00:00:00,1997-10-13 00:00:00,94.0
10952,ALFKI,1998-03-16 00:00:00,1998-01-15 00:00:00,60.0
11011,ALFKI,1998-04-09 00:00:00,1998-03-16 00:00:00,24.0
10308,ANATR,1996-09-18 00:00:00,None,None
10625,ANATR,1997-08-08 00:00:00,1996-09-18 00:00:00,324.0
10759,ANATR,1997-11-28 00:00:00,1997-08-08 00:00:00,112.0
10926,ANATR,1998-03-04 00:00:00,1997-11-28 00:00:00,96.0


#### Task 4
Calculate the moving average of the quantity of the last 3 orders for each product using window functions.

In [48]:
%%sql

SELECT
    OrderID,
    ProductID,
    Quantity,
    ROUND(AVG(Quantity) OVER (
        PARTITION BY ProductID 
        ORDER BY OrderID 
        ROWS BETWEEN 2 PRECEDING AND 0 FOLLOWING),2) AS MovingAvgQuantity
FROM
    OrderDetails
ORDER BY
    ProductID,
    OrderID;

 * sqlite:///northwind.db
Done.


OrderID,ProductID,Quantity,MovingAvgQuantity
10285,1,45,45.0
10294,1,18,31.5
10317,1,20,27.67
10348,1,15,17.67
10354,1,12,15.67
10370,1,15,14.0
10406,1,10,12.33
10413,1,24,16.33
10477,1,15,16.33
10522,1,40,26.33
